In [546]:

import random
import hashlib
# import base64
import math
import time

## $\text{Parte I}$

### Formatação Base64

In [547]:
class Base64:
    CHARSET = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"

    def b64encode(self, data: bytes) -> str:
        """Codifica uma byte string em uma string Base64."""
        binary_str = "".join(f"{byte:08b}" for byte in data)  # Converte bytes para binário
        padding = (6 - len(binary_str) % 6) % 6  # Verifica a necessidade de padding
        binary_str += "0" * padding  # Faz o padding com zeros

        # Converte cada 6 bits para caracteres Base64
        encoded = "".join(Base64.CHARSET[int(binary_str[i:i+6], 2)] for i in range(0, len(binary_str), 6))

        # Adiciona o padding '=' conforme bytes faltantes
        if padding:
            encoded += "=" * (padding // 2)

        return encoded

    def b64decode(self, encoded: str) -> bytes:
        """Decodifica uma string Base64 em sua byte string equivalente."""
        encoded = encoded.rstrip("=")  # Remove o padding '='
        
        binary_str = "".join(f"{Base64.CHARSET.index(char):06b}" for char in encoded) # Para cada caractere, converte em um binário de 6 bits

        # Converte cada 8 bits para bytes
        byte_chunks = [binary_str[i:i+8] for i in range(0, len(binary_str), 8)]
        decoded_bytes = bytes(int(chunk, 2) for chunk in byte_chunks if len(chunk) == 8)

        return decoded_bytes

In [548]:
base64 = Base64()

### Funções utilitárias

In [549]:
Key = tuple[int, int]

# def is_prime(n):
#     """Teste de primalidade de Miller-Rabin. Detalhes: https://pt.wikipedia.org/wiki/Teste_de_primalidade_de_Miller%E2%80%93Rabin"""
#     # 1 < a < n
#     a = random.randint(2, n - 1)
#     d = 0
#     # s = max{r ∈ N | 2^r divide n - 1}
#     for r in range(1, n):
#         if (n - 1) % (2 ** r) == 0:
#             # d = (n - 1) / (2^r)
#             d = (n - 1) // (2 ** r)
#         else:
#             break
        
#     # a^d ≡ 1 mod n
#     if pow(a, d, n) == 1:
#         return True
#     else:
#         return False

def is_prime_trial_division(n: int) -> bool:
    '''Testa se dado inteiro n é primo por tentativa de divisões'''
    if n == 2:
        return True
    if n < 2 or n % 2 == 0:
        return False
    for i in range(3, math.ceil(math.sqrt(n)), 2):
        if n % i == 0:
            return False
    return True

_known_primes = [2] + \
    [x for x in range(3, 1000, 2) if is_prime_trial_division(x)]

"""Teste de primalidade de Miller-Rabin. Detalhes: https://rosettacode.org/wiki/Miller%E2%80%93Rabin_primality_test#Python:_Probably_correct_answers e https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test"""
def is_prime(n, _precision_for_huge_n=16):
    def _try_composite(a, d, n, s):
        if pow(a, d, n) == 1:
            return False
        for i in range(s):
            if pow(a, 2**i * d, n) == n-1:
                return False
        return True # n  is definitely composite
 
    if n in _known_primes:
        return True
    if any((n % p) == 0 for p in _known_primes) or n in (0, 1):
        return False
    d, s = n - 1, 0
    while not d % 2:
        d, s = d >> 1, s + 1
    # Returns exact according to http://primes.utm.edu/prove/prove2_3.html
    if n < 1373653: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3))
    if n < 25326001: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5))
    if n < 118670087467: 
        if n == 3215031751: 
            return False
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7))
    if n < 2152302898747: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11))
    if n < 3474749660383: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11, 13))
    if n < 341550071728321: 
        return not any(_try_composite(a, d, n, s) for a in (2, 3, 5, 7, 11, 13, 17))
    # otherwise
    return not any(_try_composite(a, d, n, s) 
                   for a in _known_primes[:_precision_for_huge_n])


def generate_big_prime_pair(b):
    p = random.getrandbits(b)
    q = random.getrandbits(b)

    while not is_prime(p):
        p = random.getrandbits(b)
    
    while not is_prime(q) or p == q:
        q = random.getrandbits(b)

    return (p, q)   

def I2OSP(x: int, l: int):
    """Integer-to-Octet-String, responsável por converter um inteiro em bytes.
       Detalhes:"https://www.inf.pucrs.br/calazans/graduate/TPVLSI_I/RSA-oaep_spec.pdf, seção 1.1.2"""
    
    # conversão para base 256 com tamanho l
    try:
        return x.to_bytes(l, byteorder='big')
    except OverflowError:
        return x.to_bytes((l+7)//8, byteorder='big')
        

def OS2IP(X: bytes):
    """Octet-String-to-Integer, responsável por converter bytes em um inteiro
       Detalhes:"https://www.inf.pucrs.br/calazans/graduate/TPVLSI_I/RSA-oaep_spec.pdf, seção 1.1.2"""
    return int.from_bytes(X, byteorder='big')

def OS2B64(X: bytes):
    """Converte um octeto em BASE64"""
    return base64.b64encode(X)

def B642OS(x: str):
    """Converte um BASE64 em octeto"""
    return base64.b64decode(x)

def I2B64(x: int):
    """Converte um inteiro em BASE64"""
    return base64.b64encode(str(x).encode())

def B64concat(a64: str, b64: str) -> str:
    """Concatena dois BASE64"""
    _a = base64.b64decode(a64)
    _b = base64.b64decode(b64)
    return base64.b64encode(_a + _b)

def bitwise_xor(a: bytes, b: bytes):
    """Operação XOR entre dois bytes"""
    r=b""
    
    for i in range(max(len(a), len(b))):
        if i >= len(a):
            r += b[i].to_bytes(1, byteorder='big')
        elif i >= len(b):
            r += a[i].to_bytes(1, byteorder='big')
        else:
            r += (a[i] ^ b[i]).to_bytes(1, byteorder='big')
    
    return r

def key_len(key: Key):
    """Retorna o número de octetos do modulo n da chave"""
    _, n = key
    return math.ceil(n.bit_length() / 8)

### Geração de chaves

In [550]:
def generate_keypair() -> tuple[Key, Key]:
    p, q = generate_big_prime_pair(1024)
    
    n = p * q
    phi = (p - 1) * (q - 1)
    
    e = generate_public_key(phi)
    d = generate_private_key(e, phi)
    
    return ((e, n), (d, n))

def generate_public_key(phi: int):
    # A chave pública e é um número primo tal que 1 < e < φ(n) e mdc(e, φ(n)) = 1
    e = random.randrange(2, phi)
    
    while not is_prime(e):
        e = random.randrange(2, phi)
    
    return e

def generate_private_key(e: int, phi: int):
    # A chave privada d é um número tal que e*d ≡ 1 mod φ(n)
    d = pow(e, -1, phi)
    
    return d

#### RSA

In [551]:
def RSA_encode(key: Key, m: bytes) -> bytes:
    e, n = key
    
    M = OS2IP(m)
    
    C = pow(M, e, n)
    
    return I2OSP(C, key_len(key))

def RSA_decode(key: Key, c: bytes) -> bytes:
    d, n = key
    
    C = OS2IP(c)
    
    M = pow(C, d, n)
    
    return I2OSP(M, key_len(key))

#### Padding e criptografia e descriptografia
Detalhes: https://www.inf.pucrs.br/calazans/graduate/TPVLSI_I/RSA-oaep_spec.pdf, seção 1.3.

In [573]:

def MGF(Z: bytes, l: int):
    """MGF utilizando o Hash SHA-3"""
    
    # hLen denota o número de bytes da saída da função hash
    hLen = hashlib.sha3_256().digest_size
    
    if l > 2**32 * hLen:
        raise ValueError("Máscara muito longa.")
    
    T = b""
    
    for i in range(math.ceil(l / hLen)):
        # Converte i em um octeto C de tamanho 4 com a primitiva I2OSP.
        C = I2OSP(i, 4)
        # Concatena o resultado do hash SHA-3 de seed Z e C com T.
        T += hashlib.sha3_256(Z + C).digest()
    
    # A máscara M é a string consistindo dos primeiros l octetos de T.
    return T[:l]

def OAEP_encode(M: bytes, emLen: int, P: bytes = b""):
    """Encode de OAEP utilizando o Hash SHA-3"""

    hLen = hashlib.sha3_256().digest_size
    mLen = len(M)
    
    if mLen > emLen - 2 * hLen - 2:
        raise ValueError("Mensagem muito grande.")
    
    # Geração de uma string de octetos de comprimento (emLen - mLen - 2 * hLen - 2) 
    # consistindo de zeros.
    PS = b"\x00" * (emLen - mLen - 2 * hLen - 2)
    
    pHash = hashlib.sha3_256(P).digest()
    
    # Concatenação de pHash, PS, um octeto 0x01 e a mensagem M no bloco de dados DB.
    DB = pHash + PS + b"\x01" + M
    
    seed = random.getrandbits(hLen * 8).to_bytes(hLen, byteorder='big')
    
    # Mascaramento do bloco de dados
    dbMask = MGF(seed, emLen - hLen - 1)
    maskedDB = bitwise_xor(DB, dbMask)

    # Mascaramento da seed
    seedMask = MGF(maskedDB, hLen)
    maskedSeed = bitwise_xor(seed, seedMask)
    
    return b'\x00' + maskedSeed + maskedDB

def OAEP_decode(EM: bytes, P: bytes = b""):
    """Decode de OAEP utilizando o Hash SHA-3"""
    hLen = hashlib.sha3_256().digest_size
    emLen = len(EM)
    
    if emLen < 2 * hLen + 2:
        raise ValueError("Erro na decodificação. Tamanho de EM inválido.")
    
    maskedSeed = EM[1: 1 + hLen]
    maskedDB = EM[1 + hLen:]
    
    seedMask = MGF(maskedDB, hLen)
    seed = bitwise_xor(maskedSeed, seedMask)
    
    dbMask = MGF(seed, emLen - hLen - 1)
    DB = bitwise_xor(maskedDB, dbMask)
    
    pHash = hashlib.sha3_256(P).digest()
    
    # Separa a mensagem supondo o formato 00 || pHash_ || PS || 01 || M
    pHash_ = DB[:hLen]
    
    if pHash != pHash_:
        raise ValueError("Erro na decodifiocação.")
    
    # Busca o byte 01 após o padding.
    i = hLen
    while DB[i] == 0:
        i += 1
    
    if DB[i] != 1:
        raise ValueError("Byte 0x01 não encontrado.")
    
    return DB[i+1:]


#### Criptografia e descriptografia

In [553]:
def encrypt(key: Key, M64: str, P: bytes = b""):
    """"Criptografa a mensagem usando RSA-OAEP"""
    try:
        e, _ = key
        
        M = B642OS(M64)
        
        emLen = key_len(key)
        
        EM = OAEP_encode(M, emLen, P)

        C = RSA_encode(key, EM)
        
        C64 = OS2B64(C)
        
        return C64
    
    except ValueError as e:
        print(f"Erro: {e}")
        return None
    
def decrypt(key: Key,  C64: str, P: bytes = b""):
    """"Descriptografa a mensagem usando RSA-OAEP"""
    try:
        C = B642OS(C64)
        
        EM = RSA_decode(key, C)
        
        M = OAEP_decode(EM, P)
        
        M64 = OS2B64(M)
        
        return M64
    
    except ValueError as e:
        print(f"Erro: {e}")
        return None

    

#### Teste

In [554]:
public_key, private_key = generate_keypair()

print(f"Chave pública: {I2B64(public_key[0])}")
print(f"Chave privada: {I2B64(private_key[0])}")

Chave pública: NDI2NTA5OTkwNDA5NDIxMDMwMDA4NTM2MTA5Mzc4NjIyOTYzODgwNzQwMDI2OTQ4MjExNzYyMTE3NTYyNDIzNDE4MzM1NzgyOTQwNDE0OTY2OTE3Mzk2MTMzNDg4NTk3MDIxMDU5NzkxNzMzMzU2NTYxMzM2NDcwNDE1NDk1MDk4MDU4MTU1MzY0ODYzMTYwNTMyODczOTE1MzEzMTY1MTI1OTc5NTAwNzIwNjIyODUzNzMzMzQ0OTQ3NDgwOTU4ODgzODY4MDU2OTE3MTk2ODg0Njk1NzYxNjAzNDk2NDcxMDc3ODk4NTMzOTI1MTA1Njg1MDU0NzU2NDg0NTMyOTIzODIzOTYwMjc2MjA1ODY5MDI1MTE1NTE0MzM3MDQ2NjY2MzU4NDE0MTk2MTYwMDQ1NDU3MTQ2MjE5ODcxODE1ODY2NzE4Mzg5NjM3NDIxNzE1NDIwOTExNDAxMzk1NjM1MDk5NzEwODU4Mjg4NjA2NDI0NDU1ODc2MjQwNTE5MjMzMzIxNjMyNDA4MTIxNzQ2NjMyNTQ0ODI5MTU4MTA1MDc2OTI1NTgyNDUzNTY5MDg2Mzg0MTU1MTA5NTEwNTg1NzY4NDI0NjY3MjAzNTY0ODY3ODk1MDkyNTgwODc4NTcwOTUwNjkwMzM3MTI5MjcyODk3MDk5OTU5NzgwNDUyNTI3MTM3MTcwNDE1MjcxNzQ4NzAzNzM4OTg5NzY2NjQ0MTk0MDgxNDU5MjgzNzgzMjk2MTAxODIwMTcxNjQ1NDM2ODk1ODEzNzcwNjcyNDM0MTM2MDg4NjU5ODMzNTU3Nzc4MjAxNw==
Chave privada: NTUwMzc4OTk1MzU5MjkwNjE4MjIyMjM1OTgwODM4NzA3ODQ1NDA5MzkwNzUzMTYyMzYwOTM1OTQ1NjY1NjE3MjY3NDc1MTIyNjIwNjE1OTIwNTE3NjAwMDk2MjI1NDYwNTYwODMxNDI3MjQyM

In [574]:
M = OS2B64(b"Foobar supremacy!")

print(f"Mensagem original: {M}")

C = encrypt(public_key, M, b"oi")

M_ = decrypt(private_key, C, b"oi")

print(f"Mensagem cifrada: {C}")

print(f"Mensagem decifrada: {M_}")

if M == M_:
    print("Sucesso!")
else:
    print("Falha!")

Mensagem original: Rm9vYmFyIHN1cHJlbWFjeSE=
Mensagem cifrada: Bz0JvGTbbg2+r4hC+TO2lA9JW4StcVi4SdM71daDzQSnMbr88eev0YTrESbwSKDWgI2s74y8zQxN/xqdUnOjCim0cA0F9uSFyWYaqacwNjr3ov8dGKdOiDy/Dw6kX4fl6+ZX/4QmFhIcjMIzCogBZXQI5qGYzVMbZrQkQ7tOdNZg9pIVlPiYo3zHgmXHj++lIo75pde1UfCX6t0z9OjtodZCz8dcMXp3x36u7qxHI+TrOh8oPjS5//PdBXsXPkEDKwmZ0JEemg1xBkVcqRVqmjkRLCYJ0HroIQ2M+RzRq2Ke6wfxKtCxkcOy7rDLGTQP3s3x7HpZNJrV/nPHVPfY1w==
Mensagem decifrada: Rm9vYmFyIHN1cHJlbWFjeSE=
Sucesso!


## $\text{Parte II}$
### Assinatura

In [556]:

def sign(key: Key, M64: str):
    """Assinatura digital usando RSA"""
    hashedM = OS2B64(hashlib.sha3_256(B642OS(M64)).digest())

    encryptedHash = encrypt(key, hashedM)
    
    return B64concat(encryptedHash, M64)

## $\text{Parte III}$
### Verificação

In [557]:
def verify(key: Key, signedM: str):
    """Verificação da assinatura digital usando RSA"""
    emLen = key_len(key)
    
    # Como emLen indica um tamanho em bytes, essa conversão é necessária
    OSsignedM = B642OS(signedM)
    
    encryptedHash = OS2B64(OSsignedM[:emLen])

    decryptedHash = decrypt(key, encryptedHash)
    
    hashedM = OS2B64(hashlib.sha3_256(OSsignedM[emLen:]).digest())
    
    return hashedM == decryptedHash

## Criptografia + Assinatura de arquivo

In [558]:
with open("lorem.txt", "rb") as f:   
    # Byte string convertida em Base64
    M = base64.b64encode(f.read())
    signedM = sign(private_key, M)

    
    if signedM:
        with open("lorem_signed.txt", "wb") as f:
            f.write(signedM.encode())

In [559]:
with open("lorem_signed.txt", "rb") as f:  
    # Byte string tornada string (já formatada em Base64)
    C = f.read().decode()
    
    M_ = verify(public_key, C)
        
print(M_)
    


True


# Benchmark da função geradora de chaves

In [560]:
t1 = time.time()

for _ in range(100):
    generate_keypair()
    
t2 = time.time()

In [561]:
print(f"Tempo médio de execução: {((t2 - t1)/100):.1f} segundos.")

Tempo médio de execução: 17.8 segundos.
